In [ ]:
import os
import sys
import cv2
import tensorflow as tf
import numpy as np
from tensorflow import keras
from matplotlib import pyplot
from skimage import transform

In [ ]:
#return labels and ids of images
directory = 'A:/Plant_Phenotyping_Datasets/Plant/Ara2013-RPi'
def labels_and_ids():
    
    label_col = 1
    id_col = 0
    file = open('A:\Plant_Phenotyping_Datasets\Plant\Ara2013-RPi\Metadata.csv', 'r' , encoding='utf-8-sig')
    labels = []
    ids = []

    for line in file:
        line = line.rstrip()
        temp = line.split(',')
        labels.append(temp[label_col])
        ids.append(temp[id_col])
    return labels,ids


In [ ]:
#Data augmentation
def augment(image):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image , [299,299])
    image = (image/255.0)
    image = tf.image.random_brightness(image, max_delta=0.5)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.5 )
    image = tf.image.flip_left_right(image)
    #image = tf.image.central_crop(image,central_fraction=0.5)
    return image
    

In [ ]:
#loading dataset

X= []
Y = []
labels,ids = labels_and_ids()
for id in ids:
    failed = 0
    image_path = os.path.join(directory , id + '_rgb.png')
    #image_path = 'A:/Plant_Phenotyping_Datasets/Plant/Ara2013-RPi'+'/'+id+'.png'
    #print(image_path)
    
    img_arry = cv2.imread(image_path)
    image = augment(img_arry)
    #print(image)
    X.append(image)
    
for label in labels:
    Y.append(label)


In [ ]:
X= np.asarray(X)
Y= np.asarray(Y).astype('float32')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train , y_test = train_test_split(X,Y,test_size = 0.20)

x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.20)


In [ ]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

In [ ]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.optimizers import RMSprop, SGD, Adam

In [ ]:
from tensorflow import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
model = InceptionResNetV2(weights='imagenet', include_top=True, input_shape=(299, 299, 3))
x = model.get_layer(index=len(model.layers)-2).output

#print(x)
x= Dense(1)(x)
model = Model(inputs=model.input, outputs=x)
model.summary()

In [ ]:
x_train.shape

In [ ]:
from keras.optimizers import RMSprop, SGD, Adam
opt = RMSprop(lr = 0.0001)
model.compile(loss='mean_squared_error', optimizer = opt ,metrics=['mae'])

In [ ]:
network = model.fit(x_train,y_train,batch_size=15,epochs=100, verbose=1, validation_data=(x_val, y_val))

In [ ]:
model.save('C:/Users/Mohsin/JupyterNotebook/model.h5')


In [ ]:
predictTrain = model.predict(x_train, verbose=1)
predictTrain = predictTrain.reshape(predictTrain.shape[0])
#predictTrain = predictTrain.astype('int32')
print(x_train.shape)
print(predictTrain.shape)
print(y_train.shape)
print("predict train")
print(np.round(predictTrain))
print("y train")
print(y_train)

In [ ]:
predict_test = model.predict(x_val,verbose=1)
print(np.round(predict_test))